In [1]:
pip install pandas==1.5.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 75.3 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [2]:
import pandas as pd
pd.__version__

'1.5.3'

# 아파트 거래 수정(과거 수치 평균 컬럼 추가)

In [ ]:
# 아파트 거래 테이블 불러오기기
final_df2 = pd.read_pickle('/content/drive/MyDrive/house_price/final_df2.pkl')
final_df2.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,deal_to_full_rent_rate,deal_to_year_rent_rate
0,2011,1,5,강남구,개포동,138,0,89400,11000,778,8.13,114.910004
1,2011,1,5,강남구,개포동,185,0,89000,22000,1786,4.05,49.830002
2,2011,1,6,강남구,개포동,138,0,89400,7500,778,11.92,114.910004
3,2011,1,6,강남구,개포동,185,0,89000,30666,1424,2.90,62.500000
4,2011,1,7,강남구,개포동,12,0,35000,24000,1196,1.46,29.260000


In [ ]:
final_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24144106 entries, 0 to 24144105
Data columns (total 12 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   year                    int16  
 1   month                   int16  
 2   day                     int16  
 3   address_1               object 
 4   address_2               object 
 5   address_3               int16  
 6   address_4               int16  
 7   deal_price              int32  
 8   full_rent_price         int32  
 9   year_rent_price         int32  
 10  deal_to_full_rent_rate  float32
 11  deal_to_year_rent_rate  float32
dtypes: float32(2), int16(5), int32(3), object(2)
memory usage: 1.0+ GB


## 해당달의 수치들은 과거달의 평균들과 비교 

- 거래개수, 지표등은 해당달의 수치를 다음달에 알 수 있지만 체결가격들은 굳이 한달 씩 미룰 필요는 없을 듯

In [ ]:
# 지역별 달별 거래 평균을 구함함
group_final_df = final_df2.groupby(['address_1','address_2','address_3','address_4','year','month']).agg({'deal_price': 'mean','full_rent_price':'mean',
                                                                                                          'year_rent_price':'mean','deal_to_full_rent_rate':'mean','deal_to_year_rent_rate':'mean'}).copy()
group_final_df.reset_index(inplace=True)
group_final_df.head(10)

,address_1,address_2,address_3,address_4,year,month,deal_price,full_rent_price,year_rent_price,deal_to_full_rent_rate,deal_to_year_rent_rate
0,강남구,개포동,12,0,2011,1,40938.000000,18332.640000,1190.480000,2.333200,35.127602
1,강남구,개포동,12,0,2011,2,46330.357143,16232.107143,1176.464286,3.332143,40.936428
2,강남구,개포동,12,0,2011,3,47822.580645,17971.774194,1256.774194,2.725806,39.302902
3,강남구,개포동,12,0,2011,4,45638.666667,16180.866667,1026.800000,3.468000,46.846001
4,강남구,개포동,12,0,2011,5,41406.451613,18709.645161,1200.387097,2.260000,35.202583
5,강남구,개포동,12,0,2011,6,42056.666667,19296.066667,1096.966667,2.219667,40.147667
6,강남구,개포동,12,0,2011,7,42387.096774,20866.129032,1332.838710,2.108387,32.929356
7,강남구,개포동,12,0,2011,8,44316.129032,20264.516129,1228.387097,2.243548,36.480968
8,강남구,개포동,12,0,2011,9,43208.333333,20738.866667,1133.933333,2.250000,38.670666
9,강남구,개포동,12,0,2011,10,35287.096774,21646.451613,1315.354839,1.654839,28.181614


In [ ]:
import numpy as np
# 1개월 전 평균 가격들을 계산
group_final_df['deal_price_mean_1m_before'] = group_final_df['deal_price']
group_final_df['full_rent_price_mean_1m_before'] = group_final_df['full_rent_price']
group_final_df['year_rent_price_mean_1m_before'] = group_final_df['year_rent_price']
group_final_df['deal_to_full_rent_rate_mean_1m_before'] = group_final_df['deal_to_full_rent_rate']
group_final_df['deal_to_year_rent_rate_mean_1m_before'] = group_final_df['deal_to_year_rent_rate']
# 밀리면서 사라질 값들을 null 값 설정, null로 안하면 다른 지역값에 더어쓰기가 됨
group_final_df.loc[(group_final_df['year']==2022)&(group_final_df['month']==12),
                   ['deal_price_mean_1m_before', 'full_rent_price_mean_1m_before',
                    'year_rent_price_mean_1m_before','deal_to_full_rent_rate_mean_1m_before','deal_to_year_rent_rate_mean_1m_before']] = np.nan
# 한 칸씩 미뤄서 이전달의 결과를 얻음음
group_final_df[['deal_price_mean_1m_before', 'full_rent_price_mean_1m_before',
                'year_rent_price_mean_1m_before','deal_to_full_rent_rate_mean_1m_before','deal_to_year_rent_rate_mean_1m_before']] = group_final_df[['deal_price_mean_1m_before', 
                                                                     'full_rent_price_mean_1m_before','year_rent_price_mean_1m_before','deal_to_full_rent_rate_mean_1m_before',
                                                                     'deal_to_year_rent_rate_mean_1m_before']].shift(1)

# 3개월 전 평균 가격들을 계산
group_final_df['deal_price_mean_3m_before'] = group_final_df['deal_price']
group_final_df['full_rent_price_mean_3m_before'] = group_final_df['full_rent_price']
group_final_df['year_rent_price_mean_3m_before'] = group_final_df['year_rent_price']
group_final_df['deal_to_full_rent_rate_mean_3m_before'] = group_final_df['deal_to_full_rent_rate']
group_final_df['deal_to_year_rent_rate_mean_3m_before'] = group_final_df['deal_to_year_rent_rate']
for i in range(3):
    number=12-i
    group_final_df.loc[(group_final_df['year']==2022)&(group_final_df['month']==number),
                        ['deal_price_mean_3m_before', 'full_rent_price_mean_3m_before',
                         'year_rent_price_mean_3m_before','deal_to_full_rent_rate_mean_3m_before','deal_to_year_rent_rate_mean_3m_before']]=np.nan
group_final_df[['deal_price_mean_3m_before', 'full_rent_price_mean_3m_before',
                 'year_rent_price_mean_3m_before','deal_to_full_rent_rate_mean_3m_before','deal_to_year_rent_rate_mean_3m_before']] = group_final_df[['deal_price_mean_3m_before',
                 'full_rent_price_mean_3m_before','year_rent_price_mean_3m_before','deal_to_full_rent_rate_mean_3m_before','deal_to_year_rent_rate_mean_3m_before']].shift(3)

# 6개월 전 평균 가격들을 계산
group_final_df['deal_price_mean_6m_before'] = group_final_df['deal_price']
group_final_df['full_rent_price_mean_6m_before'] = group_final_df['full_rent_price']
group_final_df['year_rent_price_mean_6m_before'] = group_final_df['year_rent_price']
group_final_df['deal_to_full_rent_rate_mean_6m_before'] = group_final_df['deal_to_full_rent_rate']
group_final_df['deal_to_year_rent_rate_mean_6m_before'] = group_final_df['deal_to_year_rent_rate']
for i in range(6):
    number=12-i
    group_final_df.loc[(group_final_df['year']==2022)&(group_final_df['month']==number),
                        ['deal_price_mean_6m_before', 'full_rent_price_mean_6m_before',
                         'year_rent_price_mean_6m_before','deal_to_full_rent_rate_mean_6m_before','deal_to_year_rent_rate_mean_6m_before']]=np.nan
group_final_df[['deal_price_mean_6m_before', 'full_rent_price_mean_6m_before',
                 'year_rent_price_mean_6m_before','deal_to_full_rent_rate_mean_6m_before','deal_to_year_rent_rate_mean_6m_before']] = group_final_df[['deal_price_mean_6m_before',
                 'full_rent_price_mean_6m_before','year_rent_price_mean_6m_before','deal_to_full_rent_rate_mean_6m_before','deal_to_year_rent_rate_mean_6m_before']].shift(6)

# 12개월 전 평균 가격들을 계산
group_final_df['deal_price_mean_12m_before'] = group_final_df['deal_price']
group_final_df['full_rent_price_mean_12m_before'] = group_final_df['full_rent_price']
group_final_df['year_rent_price_mean_12m_before'] = group_final_df['year_rent_price']
group_final_df['deal_to_full_rent_rate_mean_12m_before'] = group_final_df['deal_to_full_rent_rate']
group_final_df['deal_to_year_rent_rate_mean_12m_before'] = group_final_df['deal_to_year_rent_rate']
group_final_df.loc[(group_final_df['year']==2022),
                        ['deal_price_mean_12m_before', 'full_rent_price_mean_12m_before',
                         'year_rent_price_mean_12m_before','deal_to_full_rent_rate_mean_12m_before','deal_to_year_rent_rate_mean_12m_before']]=np.nan
group_final_df[['deal_price_mean_12m_before', 'full_rent_price_mean_12m_before',
                 'year_rent_price_mean_12m_before','deal_to_full_rent_rate_mean_12m_before','deal_to_year_rent_rate_mean_12m_before']] = group_final_df[['deal_price_mean_12m_before',
                 'full_rent_price_mean_12m_before','year_rent_price_mean_12m_before','deal_to_full_rent_rate_mean_12m_before','deal_to_year_rent_rate_mean_12m_before']].shift(12)

group_final_df.drop(['deal_price','full_rent_price','year_rent_price','deal_to_full_rent_rate','deal_to_year_rent_rate'],axis=1, inplace=True)
group_final_df.dropna(inplace=True)
group_final_df.head(10)      

,address_1,address_2,address_3,address_4,year,month,deal_price_mean_1m_before,full_rent_price_mean_1m_before,year_rent_price_mean_1m_before,deal_to_full_rent_rate_mean_1m_before,...,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_to_full_rent_rate_mean_6m_before,deal_to_year_rent_rate_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before,deal_to_full_rent_rate_mean_12m_before,deal_to_year_rent_rate_mean_12m_before
12,강남구,개포동,12,0,2012,1,42426.709677,21678.483871,1155.290323,1.971290,...,42387.096774,20866.129032,1332.838710,2.108387,32.929356,40938.000000,18332.640000,1190.480000,2.333200,35.127602
13,강남구,개포동,12,0,2012,2,30774.193548,19862.774194,1112.387097,1.600968,...,44316.129032,20264.516129,1228.387097,2.243548,36.480968,46330.357143,16232.107143,1176.464286,3.332143,40.936428
14,강남구,개포동,12,0,2012,3,40358.620690,20747.275862,1088.896552,1.968621,...,43208.333333,20738.866667,1133.933333,2.250000,38.670666,47822.580645,17971.774194,1256.774194,2.725806,39.302902
15,강남구,개포동,12,0,2012,4,43870.967742,19877.935484,1057.032258,2.244194,...,35287.096774,21646.451613,1315.354839,1.654839,28.181614,45638.666667,16180.866667,1026.800000,3.468000,46.846001
16,강남구,개포동,12,0,2012,5,44583.333333,19611.533333,1185.033333,2.359333,...,53386.666667,21059.133333,1280.266667,2.613333,43.390999,41406.451613,18709.645161,1200.387097,2.260000,35.202583
17,강남구,개포동,12,0,2012,6,42716.129032,19727.129032,1055.677419,2.250968,...,42426.709677,21678.483871,1155.290323,1.971290,36.995808,42056.666667,19296.066667,1096.966667,2.219667,40.147667
18,강남구,개포동,12,0,2012,7,43401.666667,19952.733333,1163.833333,2.254333,...,30774.193548,19862.774194,1112.387097,1.600968,28.356773,42387.096774,20866.129032,1332.838710,2.108387,32.929356
19,강남구,개포동,12,0,2012,8,32402.419355,18897.806452,1070.645161,1.763226,...,40358.620690,20747.275862,1088.896552,1.968621,38.108620,44316.129032,20264.516129,1228.387097,2.243548,36.480968
20,강남구,개포동,12,0,2012,9,35403.225806,19605.354839,1282.000000,1.843226,...,43870.967742,19877.935484,1057.032258,2.244194,43.631935,43208.333333,20738.866667,1133.933333,2.250000,38.670666
21,강남구,개포동,12,0,2012,10,32816.666667,22546.366667,1250.200000,1.486667,...,44583.333333,19611.533333,1185.033333,2.359333,39.309002,35287.096774,21646.451613,1315.354839,1.654839,28.181614


In [ ]:
group_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707038 entries, 12 to 796746
Data columns (total 26 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   address_1                               707038 non-null  object 
 1   address_2                               707038 non-null  object 
 2   address_3                               707038 non-null  int64  
 3   address_4                               707038 non-null  int64  
 4   year                                    707038 non-null  int64  
 5   month                                   707038 non-null  int64  
 6   deal_price_mean_1m_before               707038 non-null  float64
 7   full_rent_price_mean_1m_before          707038 non-null  float64
 8   year_rent_price_mean_1m_before          707038 non-null  float64
 9   deal_to_full_rent_rate_mean_1m_before   707038 non-null  float32
 10  deal_to_year_rent_rate_mean_1m_before   707

In [ ]:
# 반올림 실행행
group_final_df['deal_to_full_rent_rate_mean_1m_before'] = group_final_df['deal_to_full_rent_rate_mean_1m_before'].astype(float).round(2)
group_final_df['deal_to_year_rent_rate_mean_1m_before'] = group_final_df['deal_to_year_rent_rate_mean_1m_before'].astype(float).round(2)

group_final_df['deal_to_full_rent_rate_mean_3m_before'] = group_final_df['deal_to_full_rent_rate_mean_3m_before'].astype(float).round(2)
group_final_df['deal_to_year_rent_rate_mean_3m_before'] = group_final_df['deal_to_year_rent_rate_mean_3m_before'].astype(float).round(2)

group_final_df['deal_to_full_rent_rate_mean_6m_before'] = group_final_df['deal_to_full_rent_rate_mean_6m_before'].astype(float).round(2)
group_final_df['deal_to_year_rent_rate_mean_6m_before'] = group_final_df['deal_to_year_rent_rate_mean_6m_before'].astype(float).round(2)

group_final_df['deal_to_full_rent_rate_mean_12m_before'] = group_final_df['deal_to_full_rent_rate_mean_12m_before'].astype(float).round(2)
group_final_df['deal_to_year_rent_rate_mean_12m_before'] = group_final_df['deal_to_year_rent_rate_mean_12m_before'].astype(float).round(2)

In [ ]:
group_final_df = group_final_df.astype({'address_3':'int16','address_4':'int16','year':'int16','month':'int16',
                       'deal_price_mean_1m_before':'int32', 'full_rent_price_mean_1m_before':'int32', 'year_rent_price_mean_1m_before':'int32', 'deal_to_full_rent_rate_mean_1m_before':'float32', 'deal_to_year_rent_rate_mean_1m_before':'float32',
                       'deal_price_mean_3m_before':'int32', 'full_rent_price_mean_3m_before':'int32', 'year_rent_price_mean_3m_before':'int32', 'deal_to_full_rent_rate_mean_3m_before':'float32', 'deal_to_year_rent_rate_mean_3m_before':'float32',
                       'deal_price_mean_6m_before':'int32', 'full_rent_price_mean_6m_before':'int32', 'year_rent_price_mean_6m_before':'int32', 'deal_to_full_rent_rate_mean_6m_before':'float32', 'deal_to_year_rent_rate_mean_6m_before':'float32',
                       'deal_price_mean_12m_before':'int32', 'full_rent_price_mean_12m_before':'int32', 'year_rent_price_mean_12m_before':'int32', 'deal_to_full_rent_rate_mean_12m_before':'float32', 'deal_to_year_rent_rate_mean_12m_before':'float32'})

In [ ]:
group_final_df.head()

,address_1,address_2,address_3,address_4,year,month,deal_price_mean_1m_before,full_rent_price_mean_1m_before,year_rent_price_mean_1m_before,deal_to_full_rent_rate_mean_1m_before,...,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_to_full_rent_rate_mean_6m_before,deal_to_year_rent_rate_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before,deal_to_full_rent_rate_mean_12m_before,deal_to_year_rent_rate_mean_12m_before
12,강남구,개포동,12,0,2012,1,42426,21678,1155,1.97,...,42387,20866,1332,2.11,32.930000,40938,18332,1190,2.33,35.130001
13,강남구,개포동,12,0,2012,2,30774,19862,1112,1.60,...,44316,20264,1228,2.24,36.480000,46330,16232,1176,3.33,40.939999
14,강남구,개포동,12,0,2012,3,40358,20747,1088,1.97,...,43208,20738,1133,2.25,38.669998,47822,17971,1256,2.73,39.299999
15,강남구,개포동,12,0,2012,4,43870,19877,1057,2.24,...,35287,21646,1315,1.65,28.180000,45638,16180,1026,3.47,46.849998
16,강남구,개포동,12,0,2012,5,44583,19611,1185,2.36,...,53386,21059,1280,2.61,43.389999,41406,18709,1200,2.26,35.200001


In [ ]:
group_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707038 entries, 12 to 796746
Data columns (total 26 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   address_1                               707038 non-null  object 
 1   address_2                               707038 non-null  object 
 2   address_3                               707038 non-null  int16  
 3   address_4                               707038 non-null  int16  
 4   year                                    707038 non-null  int16  
 5   month                                   707038 non-null  int16  
 6   deal_price_mean_1m_before               707038 non-null  int32  
 7   full_rent_price_mean_1m_before          707038 non-null  int32  
 8   year_rent_price_mean_1m_before          707038 non-null  int32  
 9   deal_to_full_rent_rate_mean_1m_before   707038 non-null  float32
 10  deal_to_year_rent_rate_mean_1m_before   707

In [ ]:
final_df3 = pd.merge(final_df2, group_final_df, 
         on=['address_1','address_2','address_3','address_4','year','month'], how='left')

In [ ]:
final_df3.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_to_full_rent_rate_mean_6m_before,deal_to_year_rent_rate_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before,deal_to_full_rent_rate_mean_12m_before,deal_to_year_rent_rate_mean_12m_before
0,2011,1,5,강남구,개포동,138,0,89400,11000,778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011,1,5,강남구,개포동,185,0,89000,22000,1786,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011,1,6,강남구,개포동,138,0,89400,7500,778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011,1,6,강남구,개포동,185,0,89000,30666,1424,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011,1,7,강남구,개포동,12,0,35000,24000,1196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
final_df3.isnull().sum() # 2011년 자료들이 null 값들이 됨(12개월 전 평균 가격들 때문에)

year                                            0
month                                           0
day                                             0
address_1                                       0
address_2                                       0
address_3                                       0
address_4                                       0
deal_price                                      0
full_rent_price                                 0
year_rent_price                                 0
deal_to_full_rent_rate                          0
deal_to_year_rent_rate                          0
deal_price_mean_1m_before                 2620275
full_rent_price_mean_1m_before            2620275
year_rent_price_mean_1m_before            2620275
deal_to_full_rent_rate_mean_1m_before     2620275
deal_to_year_rent_rate_mean_1m_before     2620275
deal_price_mean_3m_before                 2620275
full_rent_price_mean_3m_before            2620275
year_rent_price_mean_3m_before            2620275


In [ ]:
final_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24144106 entries, 0 to 24144105
Data columns (total 32 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   year                                    int16  
 1   month                                   int16  
 2   day                                     int16  
 3   address_1                               object 
 4   address_2                               object 
 5   address_3                               int16  
 6   address_4                               int16  
 7   deal_price                              int32  
 8   full_rent_price                         int32  
 9   year_rent_price                         int32  
 10  deal_to_full_rent_rate                  float32
 11  deal_to_year_rent_rate                  float32
 12  deal_price_mean_1m_before               float64
 13  full_rent_price_mean_1m_before          float64
 14  year_rent_price_mean_1m_before  

In [ ]:
final_df3.to_pickle('/content/drive/MyDrive/house_price/final_df3.pkl')

# 아파트 거래 수정(결측치 처리 및 변화율 계산)

In [ ]:
final_df3 = pd.read_pickle('/content/drive/MyDrive/house_price/final_df3.pkl')

In [ ]:
final_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24144106 entries, 0 to 24144105
Data columns (total 32 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   year                                    int16  
 1   month                                   int16  
 2   day                                     int16  
 3   address_1                               object 
 4   address_2                               object 
 5   address_3                               int16  
 6   address_4                               int16  
 7   deal_price                              int32  
 8   full_rent_price                         int32  
 9   year_rent_price                         int32  
 10  deal_to_full_rent_rate                  float32
 11  deal_to_year_rent_rate                  float32
 12  deal_price_mean_1m_before               float64
 13  full_rent_price_mean_1m_before          float64
 14  year_rent_price_mean_1m_before  

In [ ]:
# null 있는 row들 제거거
final_df3.dropna(inplace=True)

In [ ]:
# 변화율 = 100*((현재가격-과거가격)/과거가격)
final_df3['deal_price_mean_1m_before'] = 100*((final_df3['deal_price']-final_df3['deal_price_mean_1m_before'])/final_df3['deal_price_mean_1m_before'])
final_df3['full_rent_price_mean_1m_before'] = 100*((final_df3['full_rent_price']-final_df3['full_rent_price_mean_1m_before'])/final_df3['full_rent_price_mean_1m_before'])
final_df3['year_rent_price_mean_1m_before'] = 100*((final_df3['year_rent_price']-final_df3['year_rent_price_mean_1m_before'])/final_df3['year_rent_price_mean_1m_before'])
final_df3['deal_to_full_rent_rate_mean_1m_before'] = 100*((final_df3['deal_to_full_rent_rate']-final_df3['deal_to_full_rent_rate_mean_1m_before'])/final_df3['deal_to_full_rent_rate_mean_1m_before'])
final_df3['deal_to_year_rent_rate_mean_1m_before'] = 100*((final_df3['deal_to_year_rent_rate']-final_df3['deal_to_year_rent_rate_mean_1m_before'])/final_df3['deal_to_year_rent_rate_mean_1m_before'])

final_df3['deal_price_mean_3m_before'] = 100*((final_df3['deal_price']-final_df3['deal_price_mean_3m_before'])/final_df3['deal_price_mean_3m_before'])
final_df3['full_rent_price_mean_3m_before'] = 100*((final_df3['full_rent_price']-final_df3['full_rent_price_mean_3m_before'])/final_df3['full_rent_price_mean_3m_before'])
final_df3['year_rent_price_mean_3m_before'] = 100*((final_df3['year_rent_price']-final_df3['year_rent_price_mean_3m_before'])/final_df3['year_rent_price_mean_3m_before'])
final_df3['deal_to_full_rent_rate_mean_3m_before'] = 100*((final_df3['deal_to_full_rent_rate']-final_df3['deal_to_full_rent_rate_mean_3m_before'])/final_df3['deal_to_full_rent_rate_mean_3m_before'])
final_df3['deal_to_year_rent_rate_mean_3m_before'] = 100*((final_df3['deal_to_year_rent_rate']-final_df3['deal_to_year_rent_rate_mean_3m_before'])/final_df3['deal_to_year_rent_rate_mean_3m_before'])

final_df3['deal_price_mean_6m_before'] = 100*((final_df3['deal_price']-final_df3['deal_price_mean_6m_before'])/final_df3['deal_price_mean_6m_before'])
final_df3['full_rent_price_mean_6m_before'] = 100*((final_df3['full_rent_price']-final_df3['full_rent_price_mean_6m_before'])/final_df3['full_rent_price_mean_6m_before'])
final_df3['year_rent_price_mean_6m_before'] = 100*((final_df3['year_rent_price']-final_df3['year_rent_price_mean_6m_before'])/final_df3['year_rent_price_mean_6m_before'])
final_df3['deal_to_full_rent_rate_mean_6m_before'] = 100*((final_df3['deal_to_full_rent_rate']-final_df3['deal_to_full_rent_rate_mean_6m_before'])/final_df3['deal_to_full_rent_rate_mean_6m_before'])
final_df3['deal_to_year_rent_rate_mean_6m_before'] = 100*((final_df3['deal_to_year_rent_rate']-final_df3['deal_to_year_rent_rate_mean_6m_before'])/final_df3['deal_to_year_rent_rate_mean_6m_before'])

final_df3['deal_price_mean_12m_before'] = 100*((final_df3['deal_price']-final_df3['deal_price_mean_12m_before'])/final_df3['deal_price_mean_12m_before'])
final_df3['full_rent_price_mean_12m_before'] = 100*((final_df3['full_rent_price']-final_df3['full_rent_price_mean_12m_before'])/final_df3['full_rent_price_mean_12m_before'])
final_df3['year_rent_price_mean_12m_before'] = 100*((final_df3['year_rent_price']-final_df3['year_rent_price_mean_12m_before'])/final_df3['year_rent_price_mean_12m_before'])
final_df3['deal_to_full_rent_rate_mean_12m_before'] = 100*((final_df3['deal_to_full_rent_rate']-final_df3['deal_to_full_rent_rate_mean_12m_before'])/final_df3['deal_to_full_rent_rate_mean_12m_before'])
final_df3['deal_to_year_rent_rate_mean_12m_before'] = 100*((final_df3['deal_to_year_rent_rate']-final_df3['deal_to_year_rent_rate_mean_12m_before'])/final_df3['deal_to_year_rent_rate_mean_12m_before'])


In [ ]:
final_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21523831 entries, 4847 to 24144105
Data columns (total 32 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   year                                    int16  
 1   month                                   int16  
 2   day                                     int16  
 3   address_1                               object 
 4   address_2                               object 
 5   address_3                               int16  
 6   address_4                               int16  
 7   deal_price                              int32  
 8   full_rent_price                         int32  
 9   year_rent_price                         int32  
 10  deal_to_full_rent_rate                  float32
 11  deal_to_year_rent_rate                  float32
 12  deal_price_mean_1m_before               float64
 13  full_rent_price_mean_1m_before          float64
 14  year_rent_price_mean_1m_befor

In [ ]:
pd.set_option('display.max_columns', 35)
final_df3.head(10)

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,deal_to_full_rent_rate,deal_to_year_rent_rate,deal_price_mean_1m_before,full_rent_price_mean_1m_before,year_rent_price_mean_1m_before,deal_to_full_rent_rate_mean_1m_before,deal_to_year_rent_rate_mean_1m_before,deal_price_mean_3m_before,full_rent_price_mean_3m_before,year_rent_price_mean_3m_before,deal_to_full_rent_rate_mean_3m_before,deal_to_year_rent_rate_mean_3m_before,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_to_full_rent_rate_mean_6m_before,deal_to_year_rent_rate_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before,deal_to_full_rent_rate_mean_12m_before,deal_to_year_rent_rate_mean_12m_before
4847,2012,1,1,강남구,개포동,12,0,26500,15000,1172,1.77,22.610001,-37.538302,-30.805425,1.471861,-10.152287,-38.891891,-24.901522,-30.703132,-10.874525,7.272728,-19.765789,-37.480831,-28.112719,-12.012012,-16.113741,-31.339205,-35.267966,-18.175867,-1.512605,-24.034332,-35.639057
4849,2012,1,1,강남구,개포동,138,0,60000,12000,665,5.00,90.230003,-3.124243,49.868865,-11.920530,-36.548225,8.423458,-3.074164,46.092038,8.660131,-36.948296,-11.077164,-10.886678,48.957299,10.465116,-41.383350,-20.864756,-24.552028,26.275913,-7.638889,-44.071587,-19.916565
4850,2012,1,1,강남구,개포동,140,0,98800,25000,1170,3.95,84.440002,12.577197,20.250120,-2.743142,-8.352666,14.277986,42.490409,45.070504,33.409350,-11.235950,-28.025909,67.307334,39.828849,6.074343,-10.633484,33.713383,9.741197,64.171263,-4.723127,-41.132637,13.418409
4851,2012,1,1,강남구,개포동,141,0,96500,11500,731,8.39,132.009995,27.401149,3.912533,-12.244898,18.838531,38.505928,23.071037,2.376925,-13.079667,17.507010,39.604481,15.409914,19.072272,-0.136612,-5.941701,13.713497,32.345882,9.806168,-9.079602,18.169020,41.839470
4854,2012,1,1,강남구,개포동,185,0,80800,35500,1644,2.28,49.150002,2.031797,18.538801,1.669759,-15.241638,-0.324477,4.533223,5.188302,-19.608802,-1.724136,27.961472,7.687387,10.958305,-8.310095,-4.602516,14.782816,-1.844068,31.696097,-1.261261,-26.213593,-2.402699
4855,2012,1,1,강남구,개포동,187,0,61000,22000,2390,2.77,25.520000,-8.540242,-23.454299,18.199802,14.937754,-25.314600,-26.469702,-32.773109,38.470452,7.782103,-46.899708,-19.152828,-32.215923,58.383035,14.937754,-49.173470,31.343798,-26.095136,25.327740,72.049690,3.951125
4856,2012,1,1,강남구,개포동,189,0,63000,10000,680,6.30,92.650002,-2.264971,2.574623,-8.232119,-5.970144,5.608113,-4.109589,4.036621,-2.439024,-11.142453,-2.545491,-7.920315,0.908174,2.255639,-10.891088,-11.280283,51.602657,11.209964,-5.424200,39.689575,52.435017
4857,2012,1,1,강남구,개포동,649,0,120800,50000,3430,2.42,35.220001,0.000000,-5.515977,0.000000,4.761911,0.000000,25.496063,27.625903,2.265951,-11.678830,22.675034,-3.234592,-9.974793,9.760000,7.079650,-11.729324,21.407035,3.092784,56.621005,18.048786,-22.474134
4862,2012,1,2,강남구,개포동,12,0,26500,23500,1172,1.13,22.610001,-37.538302,8.404834,1.471861,-42.639595,-38.891891,-24.901522,8.565093,-10.874525,-31.515152,-19.765789,-37.480831,12.623406,-12.012012,-46.445496,-31.339205,-35.267966,28.191141,-1.512605,-51.502144,-35.639057
4864,2012,1,2,강남구,개포동,138,0,60000,12000,665,5.00,90.230003,-3.124243,49.868865,-11.920530,-36.548225,8.423458,-3.074164,46.092038,8.660131,-36.948296,-11.077164,-10.886678,48.957299,10.465116,-41.383350,-20.864756,-24.552028,26.275913,-7.638889,-44.071587,-19.916565


In [ ]:
# 변화율 소수점 자리 수정
final_df3['deal_to_year_rent_rate'] = final_df3['deal_to_year_rent_rate'].astype(float).round(2)

final_df3['deal_price_mean_1m_before'] = final_df3['deal_price_mean_1m_before'].astype(float).round(2)
final_df3['full_rent_price_mean_1m_before'] = final_df3['full_rent_price_mean_1m_before'].astype(float).round(2)
final_df3['year_rent_price_mean_1m_before'] = final_df3['year_rent_price_mean_1m_before'].astype(float).round(2)
final_df3['deal_to_full_rent_rate_mean_1m_before'] = final_df3['deal_to_full_rent_rate_mean_1m_before'].astype(float).round(2)
final_df3['deal_to_year_rent_rate_mean_1m_before'] = final_df3['deal_to_year_rent_rate_mean_1m_before'].astype(float).round(2)

final_df3['deal_price_mean_3m_before'] = final_df3['deal_price_mean_3m_before'].astype(float).round(2)
final_df3['full_rent_price_mean_3m_before'] = final_df3['full_rent_price_mean_3m_before'].astype(float).round(2)
final_df3['year_rent_price_mean_3m_before'] = final_df3['year_rent_price_mean_3m_before'].astype(float).round(2)
final_df3['deal_to_full_rent_rate_mean_3m_before'] = final_df3['deal_to_full_rent_rate_mean_3m_before'].astype(float).round(2)
final_df3['deal_to_year_rent_rate_mean_3m_before'] = final_df3['deal_to_year_rent_rate_mean_3m_before'].astype(float).round(2)

final_df3['deal_price_mean_6m_before'] = final_df3['deal_price_mean_6m_before'].astype(float).round(2)
final_df3['full_rent_price_mean_6m_before'] = final_df3['full_rent_price_mean_6m_before'].astype(float).round(2)
final_df3['year_rent_price_mean_6m_before'] = final_df3['year_rent_price_mean_6m_before'].astype(float).round(2)
final_df3['deal_to_full_rent_rate_mean_6m_before'] = final_df3['deal_to_full_rent_rate_mean_6m_before'].astype(float).round(2)
final_df3['deal_to_year_rent_rate_mean_6m_before'] = final_df3['deal_to_year_rent_rate_mean_6m_before'].astype(float).round(2)

final_df3['deal_price_mean_12m_before'] = final_df3['deal_price_mean_12m_before'].astype(float).round(2)
final_df3['full_rent_price_mean_12m_before'] = final_df3['full_rent_price_mean_12m_before'].astype(float).round(2)
final_df3['year_rent_price_mean_12m_before'] = final_df3['year_rent_price_mean_12m_before'].astype(float).round(2)
final_df3['deal_to_full_rent_rate_mean_12m_before'] = final_df3['deal_to_full_rent_rate_mean_12m_before'].astype(float).round(2)
final_df3['deal_to_year_rent_rate_mean_12m_before'] = final_df3['deal_to_year_rent_rate_mean_12m_before'].astype(float).round(2)
final_df3.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,deal_to_full_rent_rate,deal_to_year_rent_rate,deal_price_mean_1m_before,full_rent_price_mean_1m_before,year_rent_price_mean_1m_before,deal_to_full_rent_rate_mean_1m_before,deal_to_year_rent_rate_mean_1m_before,deal_price_mean_3m_before,full_rent_price_mean_3m_before,year_rent_price_mean_3m_before,deal_to_full_rent_rate_mean_3m_before,deal_to_year_rent_rate_mean_3m_before,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_to_full_rent_rate_mean_6m_before,deal_to_year_rent_rate_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before,deal_to_full_rent_rate_mean_12m_before,deal_to_year_rent_rate_mean_12m_before
4847,2012,1,1,강남구,개포동,12,0,26500,15000,1172,1.77,22.61,-37.54,-30.81,1.47,-10.15,-38.89,-24.90,-30.70,-10.87,7.27,-19.77,-37.48,-28.11,-12.01,-16.11,-31.34,-35.27,-18.18,-1.51,-24.03,-35.64
4849,2012,1,1,강남구,개포동,138,0,60000,12000,665,5.00,90.23,-3.12,49.87,-11.92,-36.55,8.42,-3.07,46.09,8.66,-36.95,-11.08,-10.89,48.96,10.47,-41.38,-20.86,-24.55,26.28,-7.64,-44.07,-19.92
4850,2012,1,1,강남구,개포동,140,0,98800,25000,1170,3.95,84.44,12.58,20.25,-2.74,-8.35,14.28,42.49,45.07,33.41,-11.24,-28.03,67.31,39.83,6.07,-10.63,33.71,9.74,64.17,-4.72,-41.13,13.42
4851,2012,1,1,강남구,개포동,141,0,96500,11500,731,8.39,132.01,27.40,3.91,-12.24,18.84,38.51,23.07,2.38,-13.08,17.51,39.60,15.41,19.07,-0.14,-5.94,13.71,32.35,9.81,-9.08,18.17,41.84
4854,2012,1,1,강남구,개포동,185,0,80800,35500,1644,2.28,49.15,2.03,18.54,1.67,-15.24,-0.32,4.53,5.19,-19.61,-1.72,27.96,7.69,10.96,-8.31,-4.60,14.78,-1.84,31.70,-1.26,-26.21,-2.40


In [ ]:
final_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21523831 entries, 4847 to 24144105
Data columns (total 32 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   year                                    int16  
 1   month                                   int16  
 2   day                                     int16  
 3   address_1                               object 
 4   address_2                               object 
 5   address_3                               int16  
 6   address_4                               int16  
 7   deal_price                              int32  
 8   full_rent_price                         int32  
 9   year_rent_price                         int32  
 10  deal_to_full_rent_rate                  float32
 11  deal_to_year_rent_rate                  float64
 12  deal_price_mean_1m_before               float64
 13  full_rent_price_mean_1m_before          float64
 14  year_rent_price_mean_1m_befor

In [ ]:
# float64 타입 변경경
final_df3 = final_df3.astype({'deal_to_year_rent_rate':'float32',
                              'deal_price_mean_1m_before':'float32', 'full_rent_price_mean_1m_before':'float32', 'year_rent_price_mean_1m_before':'float32','deal_to_full_rent_rate_mean_1m_before':'float32','deal_to_year_rent_rate_mean_1m_before':'float32',
                              'deal_price_mean_3m_before':'float32', 'full_rent_price_mean_3m_before':'float32', 'year_rent_price_mean_3m_before':'float32','deal_to_full_rent_rate_mean_3m_before':'float32','deal_to_year_rent_rate_mean_3m_before':'float32',
                              'deal_price_mean_6m_before':'float32', 'full_rent_price_mean_6m_before':'float32', 'year_rent_price_mean_6m_before':'float32','deal_to_full_rent_rate_mean_6m_before':'float32','deal_to_year_rent_rate_mean_6m_before':'float32',
                              'deal_price_mean_12m_before':'float32', 'full_rent_price_mean_12m_before':'float32', 'year_rent_price_mean_12m_before':'float32','deal_to_full_rent_rate_mean_12m_before':'float32','deal_to_year_rent_rate_mean_12m_before':'float32',})

In [ ]:
final_df3.head() # round 한뒤 형변환 하면 round 한것이 취소가 됨..

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,deal_to_full_rent_rate,deal_to_year_rent_rate,deal_price_mean_1m_before,full_rent_price_mean_1m_before,year_rent_price_mean_1m_before,deal_to_full_rent_rate_mean_1m_before,deal_to_year_rent_rate_mean_1m_before,deal_price_mean_3m_before,full_rent_price_mean_3m_before,year_rent_price_mean_3m_before,deal_to_full_rent_rate_mean_3m_before,deal_to_year_rent_rate_mean_3m_before,deal_price_mean_6m_before,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_to_full_rent_rate_mean_6m_before,deal_to_year_rent_rate_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before,deal_to_full_rent_rate_mean_12m_before,deal_to_year_rent_rate_mean_12m_before
4847,2012,1,1,강남구,개포동,12,0,26500,15000,1172,1.77,22.610001,-37.540001,-30.809999,1.47,-10.150000,-38.889999,-24.900000,-30.700001,-10.870000,7.270000,-19.770000,-37.480000,-28.110001,-12.01,-16.110001,-31.340000,-35.270000,-18.180000,-1.51,-24.030001,-35.639999
4849,2012,1,1,강남구,개포동,138,0,60000,12000,665,5.00,90.230003,-3.120000,49.869999,-11.92,-36.549999,8.420000,-3.070000,46.090000,8.660000,-36.950001,-11.080000,-10.890000,48.959999,10.47,-41.380001,-20.860001,-24.549999,26.280001,-7.64,-44.070000,-19.920000
4850,2012,1,1,강남구,개포동,140,0,98800,25000,1170,3.95,84.440002,12.580000,20.250000,-2.74,-8.350000,14.280000,42.490002,45.070000,33.410000,-11.240000,-28.030001,67.309998,39.830002,6.07,-10.630000,33.709999,9.740000,64.169998,-4.72,-41.130001,13.420000
4851,2012,1,1,강남구,개포동,141,0,96500,11500,731,8.39,132.009995,27.400000,3.910000,-12.24,18.840000,38.509998,23.070000,2.380000,-13.080000,17.510000,39.599998,15.410000,19.070000,-0.14,-5.940000,13.710000,32.349998,9.810000,-9.08,18.170000,41.840000
4854,2012,1,1,강남구,개포동,185,0,80800,35500,1644,2.28,49.150002,2.030000,18.540001,1.67,-15.240000,-0.320000,4.530000,5.190000,-19.610001,-1.720000,27.959999,7.690000,10.960000,-8.31,-4.600000,14.780000,-1.840000,31.700001,-1.26,-26.209999,-2.400000


In [ ]:
final_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21523831 entries, 4847 to 24144105
Data columns (total 32 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   year                                    int16  
 1   month                                   int16  
 2   day                                     int16  
 3   address_1                               object 
 4   address_2                               object 
 5   address_3                               int16  
 6   address_4                               int16  
 7   deal_price                              int32  
 8   full_rent_price                         int32  
 9   year_rent_price                         int32  
 10  deal_to_full_rent_rate                  float32
 11  deal_to_year_rent_rate                  float32
 12  deal_price_mean_1m_before               float32
 13  full_rent_price_mean_1m_before          float32
 14  year_rent_price_mean_1m_befor

In [ ]:
final_df3.to_pickle('/content/drive/MyDrive/house_price/final_df4.pkl')

# 아파트 거래 수정(미래가격 컬럼추가)

## 미래가격 데이터프레임 생성

In [3]:
# final_df2가 거래가격이0인 값들을 다 제거한 것이므로 불러옴옴
future_df = pd.read_pickle('/content/drive/MyDrive/house_price/final_df2.pkl')
future_df

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,deal_to_full_rent_rate,deal_to_year_rent_rate
0,2011,1,5,강남구,개포동,138,0,89400,11000,778,8.13,114.910004
1,2011,1,5,강남구,개포동,185,0,89000,22000,1786,4.05,49.830002
2,2011,1,6,강남구,개포동,138,0,89400,7500,778,11.92,114.910004
3,2011,1,6,강남구,개포동,185,0,89000,30666,1424,2.90,62.500000
4,2011,1,7,강남구,개포동,12,0,35000,24000,1196,1.46,29.260000
...,...,...,...,...,...,...,...,...,...,...,...,...
24144101,2022,12,31,중랑구,중화동,438,0,69500,32000,389,2.17,178.660004
24144102,2022,12,31,중랑구,중화동,450,0,91500,32500,2090,2.82,43.779999
24144103,2022,12,31,중랑구,중화동,452,0,55000,33000,1610,1.67,34.160000
24144104,2022,12,31,중랑구,중화동,453,0,85500,57750,2070,1.48,41.299999


In [4]:
future_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24144106 entries, 0 to 24144105
Data columns (total 12 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   year                    int16  
 1   month                   int16  
 2   day                     int16  
 3   address_1               object 
 4   address_2               object 
 5   address_3               int16  
 6   address_4               int16  
 7   deal_price              int32  
 8   full_rent_price         int32  
 9   year_rent_price         int32  
 10  deal_to_full_rent_rate  float32
 11  deal_to_year_rent_rate  float32
dtypes: float32(2), int16(5), int32(3), object(2)
memory usage: 1.0+ GB


In [5]:
# 날짜 컬럼 생성성
future_df = future_df.astype({'year':'str','month':'str','day':'str'})
future_df.loc[future_df["month"].str.len()==1,"month"]='0'+future_df.loc[future_df["month"].str.len()==1,"month"]
future_df.loc[future_df["day"].str.len()==1,"day"]='0'+future_df.loc[future_df["day"].str.len()==1,"day"] # 일이 있는 컬럼에서 1자리 숫자인 경우 앞에 0을 추가성
future_df['date'] = pd.to_datetime(future_df['year']+future_df['month']+future_df['day']) 
future_df = future_df.astype({'year':'int16','month':'int16','day':'int16'})
future_df.head(10)

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,deal_to_full_rent_rate,deal_to_year_rent_rate,date
0,2011,1,5,강남구,개포동,138,0,89400,11000,778,8.13,114.910004,2011-01-05
1,2011,1,5,강남구,개포동,185,0,89000,22000,1786,4.05,49.830002,2011-01-05
2,2011,1,6,강남구,개포동,138,0,89400,7500,778,11.92,114.910004,2011-01-06
3,2011,1,6,강남구,개포동,185,0,89000,30666,1424,2.90,62.500000,2011-01-06
4,2011,1,7,강남구,개포동,12,0,35000,24000,1196,1.46,29.260000,2011-01-07
5,2011,1,7,강남구,개포동,138,0,89400,7000,778,12.77,114.910004,2011-01-07
6,2011,1,7,강남구,개포동,141,0,80300,12000,1134,6.69,70.809998,2011-01-07
7,2011,1,7,강남구,개포동,185,0,79000,24000,1424,3.29,55.480000,2011-01-07
8,2011,1,8,강남구,개포동,12,0,35000,24000,1196,1.46,29.260000,2011-01-08
9,2011,1,8,강남구,개포동,138,0,89400,9000,658,9.93,135.869995,2011-01-08


In [6]:
future_df.tail()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,deal_to_full_rent_rate,deal_to_year_rent_rate,date
24144101,2022,12,31,중랑구,중화동,438,0,69500,32000,389,2.17,178.660004,2022-12-31
24144102,2022,12,31,중랑구,중화동,450,0,91500,32500,2090,2.82,43.779999,2022-12-31
24144103,2022,12,31,중랑구,중화동,452,0,55000,33000,1610,1.67,34.160000,2022-12-31
24144104,2022,12,31,중랑구,중화동,453,0,85500,57750,2070,1.48,41.299999,2022-12-31
24144105,2022,12,31,중랑구,중화동,454,0,96000,40000,2140,2.40,44.860001,2022-12-31


In [7]:
# 1년 전 날짜 구하기(데이터 프레임과 합치면 미래가격을 구할 수 있음음)
future_df['date'] = future_df['date'] - pd.Timedelta(days=365)

In [8]:
future_df

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,deal_to_full_rent_rate,deal_to_year_rent_rate,date
0,2011,1,5,강남구,개포동,138,0,89400,11000,778,8.13,114.910004,2010-01-05
1,2011,1,5,강남구,개포동,185,0,89000,22000,1786,4.05,49.830002,2010-01-05
2,2011,1,6,강남구,개포동,138,0,89400,7500,778,11.92,114.910004,2010-01-06
3,2011,1,6,강남구,개포동,185,0,89000,30666,1424,2.90,62.500000,2010-01-06
4,2011,1,7,강남구,개포동,12,0,35000,24000,1196,1.46,29.260000,2010-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24144101,2022,12,31,중랑구,중화동,438,0,69500,32000,389,2.17,178.660004,2021-12-31
24144102,2022,12,31,중랑구,중화동,450,0,91500,32500,2090,2.82,43.779999,2021-12-31
24144103,2022,12,31,중랑구,중화동,452,0,55000,33000,1610,1.67,34.160000,2021-12-31
24144104,2022,12,31,중랑구,중화동,453,0,85500,57750,2070,1.48,41.299999,2021-12-31


In [9]:
future_df['year'] = future_df['date'].dt.year
future_df['month'] = future_df['date'].dt.month
future_df['day'] = future_df['date'].dt.day
future_df

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,deal_to_full_rent_rate,deal_to_year_rent_rate,date
0,2010,1,5,강남구,개포동,138,0,89400,11000,778,8.13,114.910004,2010-01-05
1,2010,1,5,강남구,개포동,185,0,89000,22000,1786,4.05,49.830002,2010-01-05
2,2010,1,6,강남구,개포동,138,0,89400,7500,778,11.92,114.910004,2010-01-06
3,2010,1,6,강남구,개포동,185,0,89000,30666,1424,2.90,62.500000,2010-01-06
4,2010,1,7,강남구,개포동,12,0,35000,24000,1196,1.46,29.260000,2010-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24144101,2021,12,31,중랑구,중화동,438,0,69500,32000,389,2.17,178.660004,2021-12-31
24144102,2021,12,31,중랑구,중화동,450,0,91500,32500,2090,2.82,43.779999,2021-12-31
24144103,2021,12,31,중랑구,중화동,452,0,55000,33000,1610,1.67,34.160000,2021-12-31
24144104,2021,12,31,중랑구,중화동,453,0,85500,57750,2070,1.48,41.299999,2021-12-31


In [10]:
future_df.drop(["date","full_rent_price","year_rent_price","deal_to_full_rent_rate","deal_to_year_rent_rate"], axis=1,inplace=True)
future_df.columns = ['year','month','day','address_1','address_2','address_3','address_4','future_deal_price']
future_df

,year,month,day,address_1,address_2,address_3,address_4,future_deal_price
0,2010,1,5,강남구,개포동,138,0,89400
1,2010,1,5,강남구,개포동,185,0,89000
2,2010,1,6,강남구,개포동,138,0,89400
3,2010,1,6,강남구,개포동,185,0,89000
4,2010,1,7,강남구,개포동,12,0,35000
...,...,...,...,...,...,...,...,...
24144101,2021,12,31,중랑구,중화동,438,0,69500
24144102,2021,12,31,중랑구,중화동,450,0,91500
24144103,2021,12,31,중랑구,중화동,452,0,55000
24144104,2021,12,31,중랑구,중화동,453,0,85500


In [11]:
future_df = future_df.astype({'year':'int16','month':'int16','day':'int16'})
future_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24144106 entries, 0 to 24144105
Data columns (total 8 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   year               int16 
 1   month              int16 
 2   day                int16 
 3   address_1          object
 4   address_2          object
 5   address_3          int16 
 6   address_4          int16 
 7   future_deal_price  int32 
dtypes: int16(5), int32(1), object(2)
memory usage: 690.8+ MB


In [12]:
future_df.to_pickle('/content/drive/MyDrive/house_price/future_df.pkl')

# 미래가격과 final_df4 *merge*

In [3]:
# 파일들 불러오기
future_df = pd.read_pickle('/content/drive/MyDrive/house_price/future_df.pkl')
final_df4 = pd.read_pickle('/content/drive/MyDrive/house_price/final_df4.pkl')

In [4]:
final_df5 = pd.merge(final_df4, future_df, on=['address_1','address_2','address_3','address_4',
                                   'year','month','day'], how = 'inner')

In [5]:
final_df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18875832 entries, 0 to 18875831
Data columns (total 33 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   year                                    int16  
 1   month                                   int16  
 2   day                                     int16  
 3   address_1                               object 
 4   address_2                               object 
 5   address_3                               int16  
 6   address_4                               int16  
 7   deal_price                              int32  
 8   full_rent_price                         int32  
 9   year_rent_price                         int32  
 10  deal_to_full_rent_rate                  float32
 11  deal_to_year_rent_rate                  float32
 12  deal_price_mean_1m_before               float32
 13  full_rent_price_mean_1m_before          float32
 14  year_rent_price_mean_1m_before  

In [6]:
final_df5

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_to_full_rent_rate_mean_6m_before,deal_to_year_rent_rate_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before,deal_to_full_rent_rate_mean_12m_before,deal_to_year_rent_rate_mean_12m_before,future_deal_price
0,2012,1,1,강남구,개포동,12,0,26500,15000,1172,...,-28.110001,-12.01,-16.110001,-31.340000,-35.270000,-18.180000,-1.510000,-24.030001,-35.639999,40000
1,2012,1,1,강남구,개포동,138,0,60000,12000,665,...,48.959999,10.47,-41.380001,-20.860001,-24.549999,26.280001,-7.640000,-44.070000,-19.920000,45000
2,2012,1,1,강남구,개포동,140,0,98800,25000,1170,...,39.830002,6.07,-10.630000,33.709999,9.740000,64.169998,-4.720000,-41.130001,13.420000,85750
3,2012,1,1,강남구,개포동,141,0,96500,11500,731,...,19.070000,-0.14,-5.940000,13.710000,32.349998,9.810000,-9.080000,18.170000,41.840000,86500
4,2012,1,1,강남구,개포동,185,0,80800,35500,1644,...,10.960000,-8.31,-4.600000,14.780000,-1.840000,31.700001,-1.260000,-26.209999,-2.400000,49000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18875827,2021,12,31,중랑구,중화동,438,0,62500,23000,389,...,-23.330000,0.00,61.900002,23.959999,68.919998,-23.330000,0.000000,121.139999,68.910004,69500
18875828,2021,12,31,중랑구,중화동,450,0,83200,31500,1336,...,-24.850000,-13.86,44.259998,35.240002,37.320000,-9.900000,-24.090000,46.669998,80.940002,91500
18875829,2021,12,31,중랑구,중화동,452,0,55000,45000,1480,...,8.870000,0.00,-9.630000,0.000000,0.000000,36.360001,0.000000,-26.950001,0.000000,55000
18875830,2021,12,31,중랑구,중화동,453,0,80000,50000,2070,...,-9.090000,-13.53,10.340000,15.650000,19.940001,-9.090000,7.250000,32.230000,9.150000,85500


In [7]:
final_df5.isnull().sum()

year                                      0
month                                     0
day                                       0
address_1                                 0
address_2                                 0
address_3                                 0
address_4                                 0
deal_price                                0
full_rent_price                           0
year_rent_price                           0
deal_to_full_rent_rate                    0
deal_to_year_rent_rate                    0
deal_price_mean_1m_before                 0
full_rent_price_mean_1m_before            0
year_rent_price_mean_1m_before            0
deal_to_full_rent_rate_mean_1m_before     0
deal_to_year_rent_rate_mean_1m_before     0
deal_price_mean_3m_before                 0
full_rent_price_mean_3m_before            0
year_rent_price_mean_3m_before            0
deal_to_full_rent_rate_mean_3m_before     0
deal_to_year_rent_rate_mean_3m_before     0
deal_price_mean_6m_before       

In [8]:
final_df5.to_pickle('/content/drive/MyDrive/house_price/final_df5.pkl')

# 아파트 거래가격 + 거래량 생성

In [3]:
final_df5 = pd.read_pickle('/content/drive/MyDrive/house_price/final_df5.pkl')
final_df5.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,full_rent_price_mean_6m_before,year_rent_price_mean_6m_before,deal_to_full_rent_rate_mean_6m_before,deal_to_year_rent_rate_mean_6m_before,deal_price_mean_12m_before,full_rent_price_mean_12m_before,year_rent_price_mean_12m_before,deal_to_full_rent_rate_mean_12m_before,deal_to_year_rent_rate_mean_12m_before,future_deal_price
0,2012,1,1,강남구,개포동,12,0,26500,15000,1172,...,-28.110001,-12.01,-16.110001,-31.340000,-35.270000,-18.180000,-1.51,-24.030001,-35.639999,40000
1,2012,1,1,강남구,개포동,138,0,60000,12000,665,...,48.959999,10.47,-41.380001,-20.860001,-24.549999,26.280001,-7.64,-44.070000,-19.920000,45000
2,2012,1,1,강남구,개포동,140,0,98800,25000,1170,...,39.830002,6.07,-10.630000,33.709999,9.740000,64.169998,-4.72,-41.130001,13.420000,85750
3,2012,1,1,강남구,개포동,141,0,96500,11500,731,...,19.070000,-0.14,-5.940000,13.710000,32.349998,9.810000,-9.08,18.170000,41.840000,86500
4,2012,1,1,강남구,개포동,185,0,80800,35500,1644,...,10.960000,-8.31,-4.600000,14.780000,-1.840000,31.700001,-1.26,-26.209999,-2.400000,49000


In [4]:
final_df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18875832 entries, 0 to 18875831
Data columns (total 33 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   year                                    int16  
 1   month                                   int16  
 2   day                                     int16  
 3   address_1                               object 
 4   address_2                               object 
 5   address_3                               int16  
 6   address_4                               int16  
 7   deal_price                              int32  
 8   full_rent_price                         int32  
 9   year_rent_price                         int32  
 10  deal_to_full_rent_rate                  float32
 11  deal_to_year_rent_rate                  float32
 12  deal_price_mean_1m_before               float32
 13  full_rent_price_mean_1m_before          float32
 14  year_rent_price_mean_1m_before  

In [5]:
# 아파트 거래량량 테이블 불러오기기
final_count = pd.read_pickle('/content/drive/MyDrive/house_price/month_region_count.pkl')
final_count.head()

,address_1,address_2,address_3,year,month,last_month_deal_count,last_month_full_rent_count,last_month_month_rent_count,deal_count_1m_before,full_rent_count_1m_before,month_rent_count_1m_before,deal_count_3m_before,full_rent_count_3m_before,month_rent_count_3m_before,deal_count_6m_before,full_rent_count_6m_before,month_rent_count_6m_before,deal_count_12m_before,full_rent_count_12m_before,month_rent_count_12m_before
13,강남구,개포동,12,2012,2,7,33,9,-2,1,0,-1,-7,-5,3,0,-4,-9,-5,2
14,강남구,개포동,12,2012,3,7,42,9,0,9,0,3,6,4,-1,13,0,-10,4,1
15,강남구,개포동,12,2012,4,5,49,9,-2,7,0,-4,17,0,-3,21,-4,-3,3,-7
16,강남구,개포동,12,2012,5,1,35,12,-4,-14,3,-6,2,3,-7,-5,-2,-12,5,1
17,강남구,개포동,12,2012,6,6,26,8,5,-9,-4,-1,-16,-1,2,-10,3,-1,5,0


In [6]:
final_count.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 904162 entries, 13 to 993887
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   address_1                    904162 non-null  object
 1   address_2                    904162 non-null  object
 2   address_3                    904162 non-null  int16 
 3   year                         904162 non-null  int16 
 4   month                        904162 non-null  int16 
 5   last_month_deal_count        904162 non-null  int16 
 6   last_month_full_rent_count   904162 non-null  int16 
 7   last_month_month_rent_count  904162 non-null  int16 
 8   deal_count_1m_before         904162 non-null  int16 
 9   full_rent_count_1m_before    904162 non-null  int16 
 10  month_rent_count_1m_before   904162 non-null  int16 
 11  deal_count_3m_before         904162 non-null  int16 
 12  full_rent_count_3m_before    904162 non-null  int16 
 13  month_rent_co

In [7]:
final_df6 = pd.merge(final_df5, final_count, on=['year','month','address_1','address_2','address_3'])
final_df6

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,month_rent_count_1m_before,deal_count_3m_before,full_rent_count_3m_before,month_rent_count_3m_before,deal_count_6m_before,full_rent_count_6m_before,month_rent_count_6m_before,deal_count_12m_before,full_rent_count_12m_before,month_rent_count_12m_before
0,2012,2,1,강남구,개포동,12,0,50500,19000,1210,...,0,-1,-7,-5,3,0,-4,-9,-5,2
1,2012,2,2,강남구,개포동,12,0,50100,19000,1210,...,0,-1,-7,-5,3,0,-4,-9,-5,2
2,2012,2,3,강남구,개포동,12,0,50100,30000,1210,...,0,-1,-7,-5,3,0,-4,-9,-5,2
3,2012,2,4,강남구,개포동,12,0,50100,24500,1194,...,0,-1,-7,-5,3,0,-4,-9,-5,2
4,2012,2,5,강남구,개포동,12,0,50100,23000,1194,...,0,-1,-7,-5,3,0,-4,-9,-5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18853197,2021,12,27,중랑구,중화동,454,0,96000,45150,2140,...,0,-1,1,0,0,1,0,0,0,0
18853198,2021,12,28,중랑구,중화동,454,0,96000,45150,2140,...,0,-1,1,0,0,1,0,0,0,0
18853199,2021,12,29,중랑구,중화동,454,0,96000,45150,2140,...,0,-1,1,0,0,1,0,0,0,0
18853200,2021,12,30,중랑구,중화동,454,0,96000,45150,2140,...,0,-1,1,0,0,1,0,0,0,0


In [8]:
final_df6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18853202 entries, 0 to 18853201
Data columns (total 48 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   year                                    int16  
 1   month                                   int16  
 2   day                                     int16  
 3   address_1                               object 
 4   address_2                               object 
 5   address_3                               int16  
 6   address_4                               int16  
 7   deal_price                              int32  
 8   full_rent_price                         int32  
 9   year_rent_price                         int32  
 10  deal_to_full_rent_rate                  float32
 11  deal_to_year_rent_rate                  float32
 12  deal_price_mean_1m_before               float32
 13  full_rent_price_mean_1m_before          float32
 14  year_rent_price_mean_1m_before  

In [9]:
final_df6.isnull().sum()

year                                      0
month                                     0
day                                       0
address_1                                 0
address_2                                 0
address_3                                 0
address_4                                 0
deal_price                                0
full_rent_price                           0
year_rent_price                           0
deal_to_full_rent_rate                    0
deal_to_year_rent_rate                    0
deal_price_mean_1m_before                 0
full_rent_price_mean_1m_before            0
year_rent_price_mean_1m_before            0
deal_to_full_rent_rate_mean_1m_before     0
deal_to_year_rent_rate_mean_1m_before     0
deal_price_mean_3m_before                 0
full_rent_price_mean_3m_before            0
year_rent_price_mean_3m_before            0
deal_to_full_rent_rate_mean_3m_before     0
deal_to_year_rent_rate_mean_3m_before     0
deal_price_mean_6m_before       

In [10]:
final_df6.to_pickle('/content/drive/MyDrive/house_price/final_df6.pkl')

# 아파트 거래 + 거래량 + 경제지표 생성

## 용량이 크기에 slice 해서 실행행

In [1]:
pip install pandas==1.5.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 45.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [2]:
import pandas as pd
pd.__version__

'1.5.3'

In [3]:
final_df6 = pd.read_pickle('/content/drive/MyDrive/house_price/final_df6.pkl')
final_df6.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,month_rent_count_1m_before,deal_count_3m_before,full_rent_count_3m_before,month_rent_count_3m_before,deal_count_6m_before,full_rent_count_6m_before,month_rent_count_6m_before,deal_count_12m_before,full_rent_count_12m_before,month_rent_count_12m_before
0,2012,2,1,강남구,개포동,12,0,50500,19000,1210,...,0,-1,-7,-5,3,0,-4,-9,-5,2
1,2012,2,2,강남구,개포동,12,0,50100,19000,1210,...,0,-1,-7,-5,3,0,-4,-9,-5,2
2,2012,2,3,강남구,개포동,12,0,50100,30000,1210,...,0,-1,-7,-5,3,0,-4,-9,-5,2
3,2012,2,4,강남구,개포동,12,0,50100,24500,1194,...,0,-1,-7,-5,3,0,-4,-9,-5,2
4,2012,2,5,강남구,개포동,12,0,50100,23000,1194,...,0,-1,-7,-5,3,0,-4,-9,-5,2


In [4]:
# 용량이 크기에 나눠서 실행행
real_final_df_list = [final_df6.iloc[i:i+100000] for i in range(0, len(final_df6 ), 100000)]

In [5]:
len(real_final_df_list)

189

In [6]:
real_final_df_list[188]

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,month_rent_count_1m_before,deal_count_3m_before,full_rent_count_3m_before,month_rent_count_3m_before,deal_count_6m_before,full_rent_count_6m_before,month_rent_count_6m_before,deal_count_12m_before,full_rent_count_12m_before,month_rent_count_12m_before
18800000,2021,1,29,중랑구,중화동,42,0,42500,24000,605,...,0,0,3,0,-2,1,0,-1,2,0
18800001,2021,1,30,중랑구,중화동,42,0,42500,24000,605,...,0,0,3,0,-2,1,0,-1,2,0
18800002,2021,1,31,중랑구,중화동,42,0,42500,24000,605,...,0,0,3,0,-2,1,0,-1,2,0
18800003,2021,1,1,중랑구,중화동,110,26,40000,25500,1300,...,0,1,0,0,0,0,0,1,-1,0
18800004,2021,1,2,중랑구,중화동,110,26,40000,25500,1300,...,0,1,0,0,0,0,0,1,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18853197,2021,12,27,중랑구,중화동,454,0,96000,45150,2140,...,0,-1,1,0,0,1,0,0,0,0
18853198,2021,12,28,중랑구,중화동,454,0,96000,45150,2140,...,0,-1,1,0,0,1,0,0,0,0
18853199,2021,12,29,중랑구,중화동,454,0,96000,45150,2140,...,0,-1,1,0,0,1,0,0,0,0
18853200,2021,12,30,중랑구,중화동,454,0,96000,45150,2140,...,0,-1,1,0,0,1,0,0,0,0


In [7]:
final_economic2 = pd.read_pickle('/content/drive/MyDrive/house_price/final_economic2.pkl')
final_economic2.head()

,date,year,month,day,apartment_index,kospi_index,korea_rp,korea_3_year,korea_10_year,korea_10-3_year,...,us_2_year_12m_before_change_rate,us_10_year_12m_before_change_rate,us_10-2_year_12m_before_change_rate,us_10-3_year_month_12m_before_change_rate,apartment_supply_12m_before_change_rate,unsold_count_12m_before_change_rate,unsold_ratio_12m_before_change_rate,last_month_deal_count_12m_before_change_rate,last_month_full_rent_count_12m_before_change_rate,last_month_month_rent_count_12m_before_change_rate
0,2012-01-01,2012,1,1,86.8,1825.74,3.25,3.32,3.77,0.45,...,-58.901830,-43.731254,-40.395170,-42.180685,30.215154,-17.98149,-37.013084,-48.293634,-20.06323,-20.922832
1,2012-01-02,2012,1,2,86.8,1826.37,3.25,3.34,3.78,0.44,...,-58.901830,-43.821236,-40.504940,-42.274143,30.215154,-17.98149,-37.013084,-48.293634,-20.06323,-20.922832
2,2012-01-03,2012,1,3,86.8,1875.41,3.25,3.36,3.79,0.43,...,-56.905158,-41.331734,-37.907062,-39.688474,30.215154,-17.98149,-37.013084,-48.293634,-20.06323,-20.922832
3,2012-01-04,2012,1,4,86.8,1866.22,3.25,3.36,3.79,0.43,...,-57.648953,-40.563212,-36.658079,-38.391740,30.215154,-17.98149,-37.013084,-48.293634,-20.06323,-20.922832
4,2012-01-05,2012,1,5,86.8,1863.74,3.25,3.34,3.78,0.44,...,-62.853107,-42.362114,-37.096189,-40.349292,30.215154,-17.98149,-37.013084,-48.293634,-20.06323,-20.922832


In [8]:
final_economic2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4018 entries, 0 to 4017
Data columns (total 85 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                4018 non-null   object 
 1   year                                                4018 non-null   int16  
 2   month                                               4018 non-null   int16  
 3   day                                                 4018 non-null   int16  
 4   apartment_index                                     4018 non-null   float64
 5   kospi_index                                         4018 non-null   float64
 6   korea_rp                                            4018 non-null   float64
 7   korea_3_year                                        4018 non-null   float64
 8   korea_10_year                                       4018 non-null   float64
 9

### 범위를 한번에 하면 메모리 오류로 인해서 나눠서 실행(0-30,30-60,60-90,90-120,120-150,150-189)

In [9]:
temp_list = list()
num_list = [[0,30],[30,60],[60,90],[90,120],[120,150],[150,170],[170,189]]
# real_final_df1 에 사용
# for i in range(num_list[0][0],num_list[0][1]):
#   temp_list.append(pd.merge(real_final_df_list[i], final_economic2, on=['year','month','day']))
#   print(i)

# real_final_df2 에 사용
# for i in range(num_list[1][0],num_list[1][1]):
#   temp_list.append(pd.merge(real_final_df_list[i], final_economic2, on=['year','month','day']))
#   print(i)

# real_final_df3 에 사용
# for i in range(num_list[2][0],num_list[2][1]):
#   temp_list.append(pd.merge(real_final_df_list[i], final_economic2, on=['year','month','day']))
#   print(i)

# real_final_df4 에 사용
# for i in range(num_list[3][0],num_list[3][1]):
#   temp_list.append(pd.merge(real_final_df_list[i], final_economic2, on=['year','month','day']))
#   print(i)

# real_final_df5 에 사용
# for i in range(num_list[4][0],num_list[4][1]):
#   temp_list.append(pd.merge(real_final_df_list[i], final_economic2, on=['year','month','day']))
#   print(i)

# real_final_df6 에 사용
# for i in range(num_list[5][0],num_list[5][1]):
#   temp_list.append(pd.merge(real_final_df_list[i], final_economic2, on=['year','month','day']))
#   print(i)

# real_final_df7 에 사용
for i in range(num_list[6][0],num_list[6][1]):
  temp_list.append(pd.merge(real_final_df_list[i], final_economic2, on=['year','month','day']))
  print(i)


170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188


In [10]:
for i,temp_df in enumerate(temp_list):
  print(i)
  if i == 0:
    real_final_df=temp_df.copy()
  else:
    real_final_df = pd.concat([real_final_df, temp_df], axis=0)
real_final_df.reset_index(drop=True,inplace=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [11]:
real_final_df

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,us_2_year_12m_before_change_rate,us_10_year_12m_before_change_rate,us_10-2_year_12m_before_change_rate,us_10-3_year_month_12m_before_change_rate,apartment_supply_12m_before_change_rate,unsold_count_12m_before_change_rate,unsold_ratio_12m_before_change_rate,last_month_deal_count_12m_before_change_rate,last_month_full_rent_count_12m_before_change_rate,last_month_month_rent_count_12m_before_change_rate
0,2017,12,21,은평구,대조동,9,46,28500,26000,1370,...,56.968481,-2.127660,-54.802891,-44.092931,-61.385199,-74.626866,-34.291687,41.071429,-0.883899,-2.786093
1,2017,12,21,은평구,대조동,31,17,23000,15000,1316,...,56.968481,-2.127660,-54.802891,-44.092931,-61.385199,-74.626866,-34.291687,41.071429,-0.883899,-2.786093
2,2017,12,21,은평구,대조동,59,48,31400,24000,1296,...,56.968481,-2.127660,-54.802891,-44.092931,-61.385199,-74.626866,-34.291687,41.071429,-0.883899,-2.786093
3,2017,12,21,은평구,대조동,87,30,18500,18000,890,...,56.968481,-2.127660,-54.802891,-44.092931,-61.385199,-74.626866,-34.291687,41.071429,-0.883899,-2.786093
4,2017,12,21,은평구,대조동,88,69,36800,25000,1170,...,56.968481,-2.127660,-54.802891,-44.092931,-61.385199,-74.626866,-34.291687,41.071429,-0.883899,-2.786093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853197,2020,12,31,중랑구,중화동,438,0,37000,30000,389,...,-92.170592,-52.266806,127.873563,130.578512,-59.647405,-70.454545,-26.781774,-43.270484,-18.626885,55.548083
1853198,2020,12,31,중랑구,중화동,450,0,62200,27560,1760,...,-92.170592,-52.266806,127.873563,130.578512,-59.647405,-70.454545,-26.781774,-43.270484,-18.626885,55.548083
1853199,2020,12,31,중랑구,중화동,452,0,55000,33000,1480,...,-92.170592,-52.266806,127.873563,130.578512,-59.647405,-70.454545,-26.781774,-43.270484,-18.626885,55.548083
1853200,2020,12,31,중랑구,중화동,453,0,66700,55000,2394,...,-92.170592,-52.266806,127.873563,130.578512,-59.647405,-70.454545,-26.781774,-43.270484,-18.626885,55.548083


In [12]:
real_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1853202 entries, 0 to 1853201
Columns: 130 entries, year to last_month_month_rent_count_12m_before_change_rate
dtypes: float32(22), float64(76), int16(20), int32(9), object(3)
memory usage: 1.4+ GB


In [13]:
# real_final_df1 에 사용
# real_final_df.to_pickle('/content/drive/MyDrive/house_price/real_final_df1.pkl')

# real_final_df2 에 사용
# real_final_df.to_pickle('/content/drive/MyDrive/house_price/real_final_df2.pkl')

# real_final_df3 에 사용
# real_final_df.to_pickle('/content/drive/MyDrive/house_price/real_final_df3.pkl')

# real_final_df4 에 사용
# real_final_df.to_pickle('/content/drive/MyDrive/house_price/real_final_df4.pkl')

# real_final_df5 에 사용
# real_final_df.to_pickle('/content/drive/MyDrive/house_price/real_final_df5.pkl')

# real_final_df6 에 사용
# real_final_df.to_pickle('/content/drive/MyDrive/house_price/real_final_df6.pkl')

# real_final_df7 에 사용
real_final_df.to_pickle('/content/drive/MyDrive/house_price/real_final_df7.pkl')


# 잘 저장 되었는지 확인

In [3]:
test = pd.read_pickle('/content/drive/MyDrive/house_price/real_final_df1.pkl')
test.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,us_2_year_12m_before_change_rate,us_10_year_12m_before_change_rate,us_10-2_year_12m_before_change_rate,us_10-3_year_month_12m_before_change_rate,apartment_supply_12m_before_change_rate,unsold_count_12m_before_change_rate,unsold_ratio_12m_before_change_rate,last_month_deal_count_12m_before_change_rate,last_month_full_rent_count_12m_before_change_rate,last_month_month_rent_count_12m_before_change_rate
0,2012,2,1,강남구,개포동,12,0,50500,19000,1210,...,-62.644628,-46.724891,-43.321555,-46.034167,-65.892924,-16.703394,144.220895,-61.192367,-15.329118,-9.427208
1,2012,2,1,강남구,개포동,138,0,90000,7500,524,...,-62.644628,-46.724891,-43.321555,-46.034167,-65.892924,-16.703394,144.220895,-61.192367,-15.329118,-9.427208
2,2012,2,1,강남구,개포동,140,0,46250,21000,452,...,-62.644628,-46.724891,-43.321555,-46.034167,-65.892924,-16.703394,144.220895,-61.192367,-15.329118,-9.427208
3,2012,2,1,강남구,개포동,141,0,80000,16000,1074,...,-62.644628,-46.724891,-43.321555,-46.034167,-65.892924,-16.703394,144.220895,-61.192367,-15.329118,-9.427208
4,2012,2,1,강남구,개포동,179,0,76000,30000,2340,...,-62.644628,-46.724891,-43.321555,-46.034167,-65.892924,-16.703394,144.220895,-61.192367,-15.329118,-9.427208


In [4]:
test2 = pd.read_pickle('/content/drive/MyDrive/house_price/real_final_df2.pkl')
test2.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,us_2_year_12m_before_change_rate,us_10_year_12m_before_change_rate,us_10-2_year_12m_before_change_rate,us_10-3_year_month_12m_before_change_rate,apartment_supply_12m_before_change_rate,unsold_count_12m_before_change_rate,unsold_ratio_12m_before_change_rate,last_month_deal_count_12m_before_change_rate,last_month_full_rent_count_12m_before_change_rate,last_month_month_rent_count_12m_before_change_rate
0,2019,6,21,강북구,수유동,50,20,24300,16000,1076,...,-30.270355,-28.975509,-19.782062,-105.279503,-28.650138,278.723404,430.797667,-6.381174,9.479058,3.194245
1,2019,6,21,강북구,수유동,50,75,34000,22000,1043,...,-30.270355,-28.975509,-19.782062,-105.279503,-28.650138,278.723404,430.797667,-6.381174,9.479058,3.194245
2,2019,6,21,강북구,수유동,51,7,35000,20500,1814,...,-30.270355,-28.975509,-19.782062,-105.279503,-28.650138,278.723404,430.797667,-6.381174,9.479058,3.194245
3,2019,6,21,강북구,수유동,63,0,30500,20000,1192,...,-30.270355,-28.975509,-19.782062,-105.279503,-28.650138,278.723404,430.797667,-6.381174,9.479058,3.194245
4,2019,6,21,강북구,수유동,77,9,20200,13000,652,...,-30.270355,-28.975509,-19.782062,-105.279503,-28.650138,278.723404,430.797667,-6.381174,9.479058,3.194245


In [5]:
test3 = pd.read_pickle('/content/drive/MyDrive/house_price/real_final_df3.pkl')
test3.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,us_2_year_12m_before_change_rate,us_10_year_12m_before_change_rate,us_10-2_year_12m_before_change_rate,us_10-3_year_month_12m_before_change_rate,apartment_supply_12m_before_change_rate,unsold_count_12m_before_change_rate,unsold_ratio_12m_before_change_rate,last_month_deal_count_12m_before_change_rate,last_month_full_rent_count_12m_before_change_rate,last_month_month_rent_count_12m_before_change_rate
0,2021,6,2,구로구,구로동,743,35,17800,9000,656,...,-11.846061,131.295488,177.011715,193.271028,-51.53593,1.428571,109.286119,-13.564831,7.363947,34.256386
1,2021,6,2,구로구,구로동,743,60,24500,16000,770,...,-11.846061,131.295488,177.011715,193.271028,-51.53593,1.428571,109.286119,-13.564831,7.363947,34.256386
2,2021,6,2,구로구,구로동,744,3,19000,10400,718,...,-11.846061,131.295488,177.011715,193.271028,-51.53593,1.428571,109.286119,-13.564831,7.363947,34.256386
3,2021,6,2,구로구,구로동,744,15,28000,16000,716,...,-11.846061,131.295488,177.011715,193.271028,-51.53593,1.428571,109.286119,-13.564831,7.363947,34.256386
4,2021,6,2,구로구,구로동,744,18,12200,10000,640,...,-11.846061,131.295488,177.011715,193.271028,-51.53593,1.428571,109.286119,-13.564831,7.363947,34.256386


In [14]:
test4 = pd.read_pickle('/content/drive/MyDrive/house_price/real_final_df4.pkl')
test4.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,us_2_year_12m_before_change_rate,us_10_year_12m_before_change_rate,us_10-2_year_12m_before_change_rate,us_10-3_year_month_12m_before_change_rate,apartment_supply_12m_before_change_rate,unsold_count_12m_before_change_rate,unsold_ratio_12m_before_change_rate,last_month_deal_count_12m_before_change_rate,last_month_full_rent_count_12m_before_change_rate,last_month_month_rent_count_12m_before_change_rate
0,2018,11,8,동작구,노량진동,325,0,80000,35500,223,...,80.270418,38.774636,-61.180079,-20.559062,321.889192,-50.0,-88.148547,-47.653773,54.305156,31.266608
1,2018,11,8,동작구,노량진동,330,0,62000,37000,1808,...,80.270418,38.774636,-61.180079,-20.559062,321.889192,-50.0,-88.148547,-47.653773,54.305156,31.266608
2,2018,11,8,동작구,노량진동,331,0,57000,39000,1932,...,80.270418,38.774636,-61.180079,-20.559062,321.889192,-50.0,-88.148547,-47.653773,54.305156,31.266608
3,2018,11,8,동작구,노량진동,332,0,78000,50000,2240,...,80.270418,38.774636,-61.180079,-20.559062,321.889192,-50.0,-88.148547,-47.653773,54.305156,31.266608
4,2018,11,8,동작구,대방동,41,0,59000,35000,1550,...,80.270418,38.774636,-61.180079,-20.559062,321.889192,-50.0,-88.148547,-47.653773,54.305156,31.266608


In [14]:
test5 = pd.read_pickle('/content/drive/MyDrive/house_price/real_final_df5.pkl')
test5.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,us_2_year_12m_before_change_rate,us_10_year_12m_before_change_rate,us_10-2_year_12m_before_change_rate,us_10-3_year_month_12m_before_change_rate,apartment_supply_12m_before_change_rate,unsold_count_12m_before_change_rate,unsold_ratio_12m_before_change_rate,last_month_deal_count_12m_before_change_rate,last_month_full_rent_count_12m_before_change_rate,last_month_month_rent_count_12m_before_change_rate
0,2013,4,27,서초구,서초동,1641,1,89000,44000,2720,...,-17.900763,-14.042334,-13.438806,-12.686891,96.705107,82.909931,-7.013126,61.049792,4.259788,30.631333
1,2013,4,27,서초구,서초동,1641,3,78000,50000,3464,...,-17.900763,-14.042334,-13.438806,-12.686891,96.705107,82.909931,-7.013126,61.049792,4.259788,30.631333
2,2013,4,27,서초구,서초동,1641,13,56750,30000,1450,...,-17.900763,-14.042334,-13.438806,-12.686891,96.705107,82.909931,-7.013126,61.049792,4.259788,30.631333
3,2013,4,27,서초구,서초동,1642,34,120000,58000,3250,...,-17.900763,-14.042334,-13.438806,-12.686891,96.705107,82.909931,-7.013126,61.049792,4.259788,30.631333
4,2013,4,27,서초구,서초동,1644,0,55700,38000,2450,...,-17.900763,-14.042334,-13.438806,-12.686891,96.705107,82.909931,-7.013126,61.049792,4.259788,30.631333


In [12]:
test6 = pd.read_pickle('/content/drive/MyDrive/house_price/real_final_df6.pkl')
test6.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,us_2_year_12m_before_change_rate,us_10_year_12m_before_change_rate,us_10-2_year_12m_before_change_rate,us_10-3_year_month_12m_before_change_rate,apartment_supply_12m_before_change_rate,unsold_count_12m_before_change_rate,unsold_ratio_12m_before_change_rate,last_month_deal_count_12m_before_change_rate,last_month_full_rent_count_12m_before_change_rate,last_month_month_rent_count_12m_before_change_rate
0,2014,5,5,양천구,신월동,547,1,19400,17000,837,...,91.992721,49.770115,43.665307,52.607589,-83.523132,-28.00659,336.936248,-19.142602,7.851465,10.274481
1,2014,5,5,양천구,신월동,547,5,25000,14700,766,...,91.992721,49.770115,43.665307,52.607589,-83.523132,-28.00659,336.936248,-19.142602,7.851465,10.274481
2,2014,5,5,양천구,신월동,547,9,26000,19000,1254,...,91.992721,49.770115,43.665307,52.607589,-83.523132,-28.00659,336.936248,-19.142602,7.851465,10.274481
3,2014,5,5,양천구,신월동,548,1,21000,18000,191,...,91.992721,49.770115,43.665307,52.607589,-83.523132,-28.00659,336.936248,-19.142602,7.851465,10.274481
4,2014,5,5,양천구,신월동,559,5,21000,12000,952,...,91.992721,49.770115,43.665307,52.607589,-83.523132,-28.00659,336.936248,-19.142602,7.851465,10.274481


In [14]:
test7 = pd.read_pickle('/content/drive/MyDrive/house_price/real_final_df7.pkl')
test7.head()

,year,month,day,address_1,address_2,address_3,address_4,deal_price,full_rent_price,year_rent_price,...,us_2_year_12m_before_change_rate,us_10_year_12m_before_change_rate,us_10-2_year_12m_before_change_rate,us_10-3_year_month_12m_before_change_rate,apartment_supply_12m_before_change_rate,unsold_count_12m_before_change_rate,unsold_ratio_12m_before_change_rate,last_month_deal_count_12m_before_change_rate,last_month_full_rent_count_12m_before_change_rate,last_month_month_rent_count_12m_before_change_rate
0,2017,12,21,은평구,대조동,9,46,28500,26000,1370,...,56.968481,-2.12766,-54.802891,-44.092931,-61.385199,-74.626866,-34.291687,41.071429,-0.883899,-2.786093
1,2017,12,21,은평구,대조동,31,17,23000,15000,1316,...,56.968481,-2.12766,-54.802891,-44.092931,-61.385199,-74.626866,-34.291687,41.071429,-0.883899,-2.786093
2,2017,12,21,은평구,대조동,59,48,31400,24000,1296,...,56.968481,-2.12766,-54.802891,-44.092931,-61.385199,-74.626866,-34.291687,41.071429,-0.883899,-2.786093
3,2017,12,21,은평구,대조동,87,30,18500,18000,890,...,56.968481,-2.12766,-54.802891,-44.092931,-61.385199,-74.626866,-34.291687,41.071429,-0.883899,-2.786093
4,2017,12,21,은평구,대조동,88,69,36800,25000,1170,...,56.968481,-2.12766,-54.802891,-44.092931,-61.385199,-74.626866,-34.291687,41.071429,-0.883899,-2.786093
